In [43]:
from neo4j import GraphDatabase
import openrouteservice
import pandas as pd

In [44]:
URI = "bolt://localhost:7687"  
USERNAME = "neo4j"  
PASSWORD = "Tanazi369"  
DATABASE = "neo4j" 
API_KEY = "5b3ce3597851110001cf6248b2b74b27a89f4836aa6790441684f9ea"
client = openrouteservice.Client(key=API_KEY)
driver = GraphDatabase.driver(URI, auth=(USERNAME, PASSWORD))

csv_files = {
    "Vehicle": r"dataset/vehicle_updated.csv",
    "Warehouse": r"dataset/warehouse_updated.csv",
    "Customer": r"dataset/customer_updated.csv",
    "Order": r"dataset/order.csv"
}



In [45]:
def route_between_two_points(coords):
    route = client.directions(coords, profile='driving-car', format='json')
    distance = route["routes"][0]["summary"]["distance"]
    duration = route["routes"][0]["summary"]["duration"]
    return distance,duration

    

In [46]:
def route_relationship(tx, label1, id1, label2, id2, rel_type, properties):    
   
        properties_str = ", ".join(f"{key}: ${key}" for key in properties.keys()) if properties else ""     
        query = f"""
            MATCH (a:{label1} {{{label1}_ID: $id1}}), (b:{label2} {{{label2}_ID: $id2}})
            MERGE (a)-[r:{rel_type} {{ {properties_str} }}]->(b)
            """
        params = {"id1": id1, "id2": id2}    
        if properties:
            params.update(properties)
        tx.run(query, **params)


In [47]:
def generate_warehouse_links():
    with driver.session(database=DATABASE) as session:
        
        query = """
        MATCH (w:Warehouse)
        RETURN w.Warehouse_ID AS id, w.location AS location
        """
        warehouses = session.execute_read(lambda tx: [
            {"id": record["id"], "location": record["location"]}
            for record in tx.run(query)
        ])

        failed_connections = []

        
        for i in range(len(warehouses)):
            for j in range(i + 1, len(warehouses)):
                w1 = warehouses[i]
                w2 = warehouses[j]

                loc1 = (w1["location"].x, w1["location"].y)
                loc2 = (w2["location"].x, w2["location"].y)
                coords = [loc1, loc2]

                try:
                    distance, duration = route_between_two_points(coords)
                    distance_km = round(distance / 1000, 4)
                    duration_hr = round(duration / 3600, 4)

                   
                    session.execute_write(
                        route_relationship,
                        "Warehouse", w1["id"],
                        "Warehouse", w2["id"],
                        "EXTERNAL", {"distance": distance_km, "duration": duration_hr}
                    )
                    session.execute_write(
                        route_relationship,
                        "Warehouse", w2["id"],
                        "Warehouse", w1["id"],
                        "EXTERNAL", {"distance": distance_km, "duration": duration_hr}
                    )

                    print(f"Connected Warehouse {w1['id']} <--> Warehouse {w2['id']} [Distance: {distance_km}km, Duration: {duration_hr}hr]")

                except Exception as e:
                    print(f"Failed to connect {w1['id']} <--> {w2['id']}: {str(e)}")
                    failed_connections.append((w1["id"], w2["id"]))

        print()
        print(failed_connections)


In [48]:
def get_orders_from_warehouse(warehouse_id):    
    with driver.session(database=DATABASE) as session:
        result = session.execute_read(lambda tx: tx.run("""
            MATCH (o:Order)-[:SHIPPED_FROM]->(w:Warehouse {Warehouse_ID: $wid})
            WHERE o.Status = 'Pending'
            RETURN o.Order_ID AS order_id, o
        """, wid=warehouse_id).data())

        if result:
            return [{"order_id": record["order_id"], "order_data": dict(record["o"])} for record in result]
        else:
            return None


In [49]:
def get_all_pending_orders():
    with driver.session(database=DATABASE) as session:
        pending_orders = session.execute_read(lambda tx: [
            {
                "order_id": record["order_id"], 
                "warehouse_id": record["warehouse_id"],
                "customer_id": record["customer_id"],
                "customer_location": record["customer_location"]
            }
            for record in tx.run("""
                MATCH (o:Order)-[:SHIPPED_FROM]->(w:Warehouse), (o)-[:PLACED_BY]->(c:Customer)
                WHERE o.Status = 'Pending'
                RETURN o.Order_ID AS order_id, 
                       w.Warehouse_ID AS warehouse_id,
                       c.Customer_ID AS customer_id,
                       c.location AS customer_location
            """)
        ])
        
        if not pending_orders:            
            return None
        
        return pending_orders


In [170]:
def get_nearest_warehouse_by_external_distance(warehouse_id):
  
    pending_orders = get_all_pending_orders()
    
    if not pending_orders:
        return None
    
    warehouses_with_orders = set(order["warehouse_id"] for order in pending_orders)

    with driver.session(database=DATABASE) as session:
        result = session.execute_read(lambda tx: tx.run("""
            MATCH (w1:Warehouse {Warehouse_ID: $wid})-[r:EXTERNAL]->(w2:Warehouse)
            WHERE w2.Warehouse_ID IN $valid_targets
            RETURN w2.Warehouse_ID AS neighbor_id, 
                   r.distance AS distance, 
                   r.duration AS duration
            ORDER BY r.distance ASC
            LIMIT 1
        """, wid=warehouse_id, valid_targets=list(warehouses_with_orders)).single())

        if result:
            return {
                "warehouse_id": result["neighbor_id"],
                "distance_km": result["distance"],
                "duration_hr": result["duration"]
            }
        else:
            return None  

In [165]:
get_nearest_warehouse_by_external_distance('W003')

In [168]:
def update_current_warehouse(vehicles):
    
    for vehicle in vehicles:
        vehicle_id = vehicle["vehicle_id"]
        current_warehouse_id = vehicle["current_warehouse"]

        if get_orders_from_warehouse(current_warehouse_id) is None:
            best_warehouse = get_nearest_warehouse_by_external_distance(current_warehouse_id)
            
            if best_warehouse is not None:
                best_warehouse_id = best_warehouse["warehouse_id"]
                distance_km = round(best_warehouse["distance_km"], 4)
                duration_hr = round(best_warehouse["duration_hr"], 4)

                with driver.session(database=DATABASE) as session:
                    vehicle_status = session.execute_read(lambda tx: tx.run("""
                         MATCH ()-[r:ROUTE_TO {vehicle_id: $vid}]->()
                         RETURN coalesce(max(r.order), 0) AS max_order,
                 coalesce(max(r.time_taken), 0) AS max_time
                    """, vid=vehicle_id).single())

                    order_counter = vehicle_status["max_order"] + 1 
                    cumulative_duration = vehicle_status["max_time"] + duration_hr 

                    session.execute_write(
                        route_relationship,
                        "Warehouse", current_warehouse_id,
                        "Warehouse", best_warehouse_id,
                        "ROUTE_TO",
                        {
                            "distance": distance_km,
                            "duration": duration_hr,
                            "order": order_counter,
                            "vehicle_id": vehicle_id,
                            "time_taken": round(cumulative_duration, 4),
                            "repositioning": True
                        }
                    )

                    print(f"Vehicle {vehicle_id} repositioned from Warehouse {current_warehouse_id} to {best_warehouse_id} [Distance: {distance_km}km | Time: {duration_hr}hr]")


                    vehicle["current_warehouse"] = best_warehouse_id
    
    return vehicles

In [52]:
def create_in_cluster_edges_for_order_group(customers):
    from collections import defaultdict

    with driver.session(database=DATABASE) as session:
        failed_connections = []

        for i in range(len(customers)):
            for j in range(i + 1, len(customers)):
                cust1 = customers[i]
                cust2 = customers[j]

                coords1 = (cust1["location"].x, cust1["location"].y)
                coords2 = (cust2["location"].x, cust2["location"].y)
                coords = [coords1, coords2]

                try:
                    distance, duration = route_between_two_points(coords)
                    distance_km = round(distance / 1000, 4)
                    duration_hr = round(duration / 3600, 4)

                    session.execute_write(
                        route_relationship, "Customer", cust1["customer_id"], "Customer", cust2["customer_id"],
                        "IN_CLUSTER", {"distance": distance_km, "duration": duration_hr}
                    )
                    session.execute_write(
                        route_relationship, "Customer", cust2["customer_id"], "Customer", cust1["customer_id"],
                        "IN_CLUSTER", {"distance": distance_km, "duration": duration_hr}
                    )

                    print(f"IN_CLUSTER: C{cust1['customer_id']} <-> C{cust2['customer_id']} [{distance_km}km, {duration_hr}hr]")

                except Exception as e:
                    print(f"Failed IN_CLUSTER: C{cust1['customer_id']} <-> C{cust2['customer_id']}: {str(e)}")
                    failed_connections.append({
                        "customer1": cust1["customer_id"],
                        "customer2": cust2["customer_id"]
                    })

        return failed_connections


In [113]:
def get_edge_data(session, from_id, to_id, is_from_warehouse=False,to_warehouse=False):
        
        def get_node_location(session, node_id, is_warehouse):
            label = "Warehouse" if is_warehouse else "Customer"
            id_field = "Warehouse_ID" if is_warehouse else "Customer_ID"

            query = f"""
                MATCH (n:{label} {{{id_field}: $id}})
                RETURN n.location AS location
            """
            result = session.run(query, {"id": node_id})
            record = result.single()

            if record and record["location"]:
                loc = record["location"]
                return [loc.longitude, loc.latitude] 
            return None
       
        if is_from_warehouse:
            query = """
                MATCH (:Warehouse {Warehouse_ID: $from})-[r:DELIVERS_TO]->(:Customer {Customer_ID: $to})
                RETURN r.distance AS dist, r.duration AS dur
            """
            
        elif to_warehouse:
            query = """
                MATCH (:Warehouse {Warehouse_ID: $to})-[r:DELIVERS_TO]->(:Customer {Customer_ID: $from})
                RETURN r.distance AS dist, r.duration AS dur
            """
        else:
            query = """
                MATCH (:Customer {Customer_ID: $from})-[r:IN_CLUSTER]->(:Customer {Customer_ID: $to})
                RETURN r.distance AS dist, r.duration AS dur
            """
        result = session.run(query, {"from": from_id, "to": to_id})
        record = result.single()
        
        if record and record["dist"] != float("inf"):
            return record["dist"], record["dur"]

    
        from_coords = get_node_location(session, from_id, is_from_warehouse or not to_warehouse)
        to_coords = get_node_location(session, to_id, to_warehouse)

        if from_coords is None or to_coords is None:
             return float("inf"), float("inf")

    
        dis,dur=route_between_two_points([from_coords, to_coords])
        dis=round(dis/1000,4)
        dur=round(dur/3600,4)
        return dis,dur

In [100]:
def get_clustered_orders_around_closest(vehicle_id, current_warehouse_id, cluster_threshold=30):
    with driver.session(database=DATABASE) as session:
        
        closest_order = session.execute_read(lambda tx: tx.run("""
            MATCH (w:Warehouse {Warehouse_ID: $wid})<-[:SHIPPED_FROM]-(o:Order)-[:PLACED_BY]->(c:Customer),
                  (w)-[r:DELIVERS_TO]->(c)
            WHERE o.Status = 'Pending'
            RETURN o.Order_ID AS order_id,
                   w.Warehouse_ID AS warehouse_id,
                   c.Customer_ID AS customer_id,
                   c.location AS customer_location,
                   r.distance AS delivery_distance
            ORDER BY r.distance ASC
            LIMIT 1
        """, wid=current_warehouse_id).single())

        if not closest_order:
            return None 

        customer_id = closest_order["customer_id"]
        
        
        clustered_orders = session.execute_read(lambda tx: [
            {
                "vehicle_id": vehicle_id,
                "order_id": record["order_id"],
                "warehouse_id": record["warehouse_id"],
                "customer_id": record["customer_id"],
                "customer_location": record["customer_location"],
                "distance_km": record["distance_to_center"]
            }
            for record in tx.run("""
                MATCH (w:Warehouse {Warehouse_ID: $wid})<-[:SHIPPED_FROM]-(o:Order)-[:PLACED_BY]->(c:Customer),
                      (wc:Customer {Customer_ID: $center_id}),
                      (c)<-[r:DELIVERS_TO]-(w)
                WHERE o.Status = 'Pending'
                WITH o, w, c, point.distance(c.location, wc.location)/1000 AS distance_to_center
                WHERE distance_to_center <= $threshold
                RETURN o.Order_ID AS order_id,
                       w.Warehouse_ID AS warehouse_id,
                       c.Customer_ID AS customer_id,
                       c.location AS customer_location,
                       distance_to_center
                ORDER BY distance_to_center ASC
            """, wid=current_warehouse_id, center_id=customer_id, threshold=cluster_threshold)
        ])
        
        if clustered_orders:
            
            customers = [
                {"customer_id": o["customer_id"], "location": o["customer_location"]}
                for o in clustered_orders
            ]
            
            create_in_cluster_edges_for_order_group(customers)
       
        return clustered_orders if clustered_orders else None


In [97]:
get_clustered_orders_around_closest("V001","W005")

C015
[{'customer_id': 'C015', 'location': POINT(76.6883 12.2667)}]


[{'vehicle_id': 'V001',
  'order_id': 'O015',
  'warehouse_id': 'W005',
  'customer_id': 'C015',
  'customer_location': POINT(76.6883 12.2667),
  'distance_km': 0.0}]

In [145]:
def tsp(vehicle,cluster_threshold=30):
    vehicle_id=vehicle['vehicle_id']
    current_warehouse_id=vehicle['current_warehouse']
    
    
    with driver.session(database=DATABASE) as session:
        
        order_result = session.run("""
           MATCH ()-[r:ROUTE_TO {vehicle_id: $vid}]->()
           RETURN coalesce(max(r.order), 0) AS max_order,
                 coalesce(max(r.time_taken), 0) AS max_time
            """, vid=vehicle_id)

        vehicle_data = order_result.single()
        order_counter = vehicle_data["max_order"] + 1  
        cumulative_duration = vehicle_data["max_time"] if vehicle_data else 0 
        
        clustered_orders = get_clustered_orders_around_closest(vehicle_id, current_warehouse_id, cluster_threshold)
        
        
           
        if not clustered_orders:
                return vehicle_data  

        customers = [order["customer_id"] for order in clustered_orders]
        
        unvisited = set(customers)
        route = []
        current = current_warehouse_id
        is_warehouse = True

        while unvisited:
            best_next = None
            min_dist = float('inf')
            for candidate in unvisited:
                dist, _ = get_edge_data(session, current, candidate, is_warehouse)
                if dist < min_dist:
                    min_dist = dist
                    best_next = candidate
            
            if best_next is None:
                break
            route.append(best_next)
            unvisited.remove(best_next)
            current = best_next
            is_warehouse = False
    

       
        last_customer = route[-1] if route else None
        return_warehouse_id = current_warehouse_id  

        if last_customer:
            wh_result = session.run("""
               MATCH (c:Customer {Customer_ID: $cid}), (w:Warehouse)
               RETURN w.Warehouse_ID AS wid, 
               point.distance(c.location, w.location) AS dist
               ORDER BY dist ASC
               LIMIT 1
            """, cid=last_customer)

            closest = wh_result.single()
            if closest:
                return_warehouse_id = closest["wid"]

        full_route = [("Warehouse", current_warehouse_id)] + [("Customer", cid) for cid in route] + [("Warehouse", return_warehouse_id)]
       
        print()
        print(vehicle_id,': ')
        print(full_route)
        print()
        for i in range(len(full_route) - 1):
                label1, id1 = full_route[i]
                label2, id2 = full_route[i + 1]
                is_from_warehouse = label1 == "Warehouse"
                to_warehouse = label2 == "Warehouse"
                
                dist, dur = get_edge_data(session, id1, id2, is_from_warehouse, to_warehouse)
                if dist == float('inf') or dur == float('inf'):
                    continue

                duration_hours = dur
                cumulative_duration += duration_hours

                
                session.execute_write(
                    route_relationship,
                    label1, id1,
                    label2, id2,
                    "ROUTE_TO",
                    {
                        "distance": round(dist, 4),
                        "duration": round(duration_hours, 4),
                        "order": order_counter,
                        "vehicle_id": vehicle_id,
                        "time_taken": round(cumulative_duration, 4)
                    }
                )

                if label2 == "Customer":
                    
                    session.run("""
                        MATCH (o:Order)-[:PLACED_BY]->(c:Customer {Customer_ID: $cust_id})
                        SET o.Status = 'scheduled',
                            o.time_taken = $time_taken
                    """, {
                        "cust_id": id2,
                        "time_taken": round(cumulative_duration, 4)
                    })

                print(f"{label1} {id1} -> {label2} {id2} [Vehicle: {vehicle_id}]")
                order_counter += 1

        session.run("""
            MATCH ()-[r:IN_CLUSTER]->()
            DELETE r
        """)
        
    vehicle['current_warehouse']= return_warehouse_id
    vehicle['time_taken']=round(cumulative_duration, 4)
    return vehicle
        

In [118]:
with driver.session(database=DATABASE) as session:   
    print(get_edge_data(session, 'C015', 'W003', False,True))

(8.9863, 0.1849)


In [143]:
def route_external_deliveries():
   
    with driver.session(database=DATABASE) as session:     
        
        vehicles = session.execute_read(lambda tx: [
            {"vehicle_id": record["vehicle_id"], 
             "current_warehouse": record["warehouse_id"],
             "time_taken": record["time_taken"],
             "active": True} 
            for record in tx.run("""
                MATCH (v:Vehicle)-[r:ROUTE_TO]->(w:Warehouse)
                WITH v, MAX(r.order) AS max_order
                MATCH (v)-[r2:ROUTE_TO]->(w:Warehouse)
                WHERE r2.order = max_order
                RETURN v.Vehicle_ID AS vehicle_id, w.Warehouse_ID AS warehouse_id,r2.time_taken AS time_taken
            """)
        ])      
       
        warehouses = session.execute_read(lambda tx: [
            {"warehouse_id": record["warehouse_id"], 
             "location": record["location"]}
            for record in tx.run("""
                MATCH (w:Warehouse)
                RETURN w.Warehouse_ID AS warehouse_id, w.location AS location
            """)
        ])
        
        iteration = 0
        while True:
            iteration += 1
            print(f"\n=== Iteration {iteration} ===")
            
            pending_orders=get_all_pending_orders()            
            if pending_orders is None:
                print("No more pending orders - routing complete!")
                break           
            print(f"Remaining pending orders: {len(pending_orders)}")
            
            vehicles=update_current_warehouse(vehicles)
            print(vehicles)
            
            vehicles.sort(key=lambda v: v.get("time_taken", 0))
            for vehicle_data in vehicles:
                updated_vehicle_data = tsp(vehicle_data)
                vehicle_data.update(updated_vehicle_data)
            
            


In [179]:
import concurrent.futures

def route_external_deliveries_parallelized():
    with driver.session(database=DATABASE) as session:     
        
        vehicles = session.execute_read(lambda tx: [
            {"vehicle_id": record["vehicle_id"], 
             "current_warehouse": record["warehouse_id"],
             "time_taken": record["time_taken"],
             "active": True} 
            for record in tx.run(""" 
                MATCH (v:Vehicle)-[r:ROUTE_TO]->(w:Warehouse)
                WITH v, MAX(r.order) AS max_order
                MATCH (v)-[r2:ROUTE_TO]->(w:Warehouse)
                WHERE r2.order = max_order
                RETURN v.Vehicle_ID AS vehicle_id, w.Warehouse_ID AS warehouse_id, r2.time_taken AS time_taken
            """)
        ])      
        
        
        warehouses = session.execute_read(lambda tx: [
            {"warehouse_id": record["warehouse_id"], 
             "location": record["location"]}
            for record in tx.run(""" 
                MATCH (w:Warehouse)
                RETURN w.Warehouse_ID AS warehouse_id, w.location AS location
            """)
        ])
        
        iteration = 0
        while True:
            iteration += 1
            print(f"\n=== Iteration {iteration} ===")
            
            pending_orders = get_all_pending_orders()
            
            if pending_orders is None:
                print("No more pending orders - routing complete!")
                break
            
            print(f"Remaining pending orders: {len(pending_orders)}")
            
           
            vehicles = update_current_warehouse(vehicles)
            print(vehicles)
            
            
            vehicles.sort(key=lambda v: v.get("time_taken", 0))
            
            
            with concurrent.futures.ThreadPoolExecutor() as executor:
                
                results = list(executor.map(tsp, vehicles))
            
            
            for vehicle_data, updated_vehicle_data in zip(vehicles, results):
                vehicle_data.update(updated_vehicle_data)

            

In [ ]:
route_external_deliveries()


=== Iteration 1 ===
Remaining pending orders: 23
[{'vehicle_id': 'V001', 'current_warehouse': 'W005', 'time_taken': 0.08, 'active': True}, {'vehicle_id': 'V002', 'current_warehouse': 'W001', 'time_taken': 0.22, 'active': True}, {'vehicle_id': 'V003', 'current_warehouse': 'W002', 'time_taken': 0.1, 'active': True}, {'vehicle_id': 'V004', 'current_warehouse': 'W003', 'time_taken': 0.09, 'active': True}, {'vehicle_id': 'V005', 'current_warehouse': 'W004', 'time_taken': 0.35, 'active': True}]

V001 : 
[('Warehouse', 'W005'), ('Customer', 'C015'), ('Warehouse', 'W003')]

Warehouse W005 -> Customer C015 [Vehicle: V001]
Customer C015 -> Warehouse W003 [Vehicle: V001]

V003 : 
[('Warehouse', 'W002'), ('Customer', 'C019'), ('Warehouse', 'W002')]


V004 : 
[('Warehouse', 'W003'), ('Customer', 'C016'), ('Warehouse', 'W001')]

Warehouse W002 -> Customer C019 [Vehicle: V003]
Warehouse W003 -> Customer C016 [Vehicle: V004]
Customer C019 -> Warehouse W002 [Vehicle: V003]
IN_CLUSTER: CC025 <-> CC006 

c:\Users\Tanaz\AppData\Local\Programs\Python\Python310\lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,


IN_CLUSTER: CC027 <-> CC018 [8.0664km, 0.2415hr]
IN_CLUSTER: CC027 <-> CC018 [8.0664km, 0.2415hr]


c:\Users\Tanaz\AppData\Local\Programs\Python\Python310\lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
c:\Users\Tanaz\AppData\Local\Programs\Python\Python310\lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
c:\Users\Tanaz\AppData\Local\Programs\Python\Python310\lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
c:\Users\Tanaz\AppData\Local\Programs\Python\Python310\lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 5th time.
  warnings.warn('Rate limit exceeded. Retrying for 

IN_CLUSTER: CC027 <-> CC020 [23.5164km, 0.5145hr]
IN_CLUSTER: CC018 <-> CC020 [31.6568km, 0.6843hr]

V002 : 
[('Warehouse', 'W002'), ('Customer', 'C027'), ('Customer', 'C018'), ('Customer', 'C020'), ('Warehouse', 'W001')]

Warehouse W002 -> Customer C027 [Vehicle: V002]
Customer C027 -> Customer C018 [Vehicle: V002]
Customer C018 -> Customer C020 [Vehicle: V002]
Customer C020 -> Warehouse W001 [Vehicle: V002]
Customer C027 -> Warehouse W001 [Vehicle: V005]
IN_CLUSTER: CC027 <-> CC020 [23.5164km, 0.5145hr]
IN_CLUSTER: CC018 <-> CC020 [31.6568km, 0.6843hr]

V001 : 
[('Warehouse', 'W002'), ('Customer', 'C027'), ('Customer', 'C020'), ('Customer', 'C018'), ('Warehouse', 'W001')]

Warehouse W002 -> Customer C027 [Vehicle: V001]
Customer C027 -> Customer C020 [Vehicle: V001]
Customer C020 -> Customer C018 [Vehicle: V001]
Customer C018 -> Warehouse W001 [Vehicle: V001]
Customer C027 -> Warehouse W001 [Vehicle: V003]

=== Iteration 3 ===
Remaining pending orders: 12
Vehicle V004 repositioned fr

c:\Users\Tanaz\AppData\Local\Programs\Python\Python310\lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 9th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,


Customer C002 -> Warehouse W001 [Vehicle: V003]

=== Iteration 8 ===
No more pending orders - routing complete!


In [149]:
def print_vehicle_route(vehicle_id):
    with driver.session(database=DATABASE) as session:
        result = session.run("""
            MATCH (from)-[r:ROUTE_TO]->(to)
            WHERE r.vehicle_id = $vehicle_id
            RETURN 
                r.order AS order, 
                from AS from_node, 
                to AS to_node, 
                r.distance AS distance, 
                r.duration AS duration
            ORDER BY r.order ASC
        """, vehicle_id=vehicle_id)

        print(f"\nRoute for Vehicle {vehicle_id}:")
        for record in result:
            order = record["order"]
            distance = record["distance"]
            duration = record["duration"]
            from_node = record["from_node"]
            to_node = record["to_node"]

            from_label = list(from_node.labels)[0]
            to_label = list(to_node.labels)[0]

            from_id = from_node.get("Warehouse_ID") if from_label == "Warehouse" else from_node.get("Customer_ID", "N/A")
            to_id = to_node.get("Warehouse_ID") if to_label == "Warehouse" else to_node.get("Customer_ID", "N/A")

            print(f"Order {order}: {from_label} {from_id} → {to_label} {to_id} | Distance: {distance} km | Duration: {duration} hr")


In [178]:
print_vehicle_route('V004')


Route for Vehicle V004:
Order 1: Vehicle N/A → Warehouse W003 | Distance: 4.54 km | Duration: 0.09 hr
Order 2: Warehouse W003 → Customer C021 | Distance: 1.6676 km | Duration: 0.0482 hr
Order 3: Customer C021 → Customer C003 | Distance: 24.9651 km | Duration: 0.4575 hr
Order 4: Customer C003 → Warehouse W003 | Distance: 25.803 km | Duration: 0.4691 hr
Order 5: Warehouse W003 → Customer C016 | Distance: 461.1426 km | Duration: 6.0362 hr
Order 6: Customer C016 → Warehouse W001 | Distance: 22.2605 km | Duration: 0.4706 hr
Order 7: Warehouse W001 → Customer C012 | Distance: 467.0068 km | Duration: 6.0661 hr
Order 8: Customer C012 → Customer C007 | Distance: 13.1694 km | Duration: 0.2345 hr
Order 9: Customer C007 → Warehouse W003 | Distance: 13.381 km | Duration: 0.244 hr
Order 10: Warehouse W003 → Warehouse W005 | Distance: 196.258 km | Duration: 3.1397 hr
Order 11: Warehouse W005 → Customer C017 | Distance: 333.8207 km | Duration: 4.224 hr
Order 12: Customer C017 → Customer C014 | Distan

In [180]:
generate_warehouse_links()

Connected Warehouse W001 <--> Warehouse W002 [Distance: 325.2071km, Duration: 4.2433hr]
Connected Warehouse W001 <--> Warehouse W003 [Distance: 467.0123km, Duration: 6.0665hr]
Connected Warehouse W001 <--> Warehouse W004 [Distance: 649.9965km, Duration: 8.3643hr]
Connected Warehouse W001 <--> Warehouse W005 [Distance: 501.1752km, Duration: 6.2701hr]
Connected Warehouse W002 <--> Warehouse W003 [Distance: 144.8403km, Duration: 1.8954hr]
Connected Warehouse W002 <--> Warehouse W004 [Distance: 572.0668km, Duration: 7.4968hr]
Connected Warehouse W002 <--> Warehouse W005 [Distance: 331.6492km, Duration: 4.3704hr]
Connected Warehouse W003 <--> Warehouse W004 [Distance: 731.4305km, Duration: 9.0601hr]
Connected Warehouse W003 <--> Warehouse W005 [Distance: 196.258km, Duration: 3.1397hr]
Connected Warehouse W004 <--> Warehouse W005 [Distance: 904.1274km, Duration: 11.4156hr]

[]
